# Orbit Propagation Validation Plotting for RMS differences

This tutorial demonstrates how to perform orbit propagation using the propagated orbit mode.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import sys

import plotly.graph_objs as go

from ostk.mathematics.objects import RealInterval
from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics.units import Mass
from ostk.physics.units import Length
from ostk.physics.units import Derived
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth
from ostk.physics.environment.objects.celestial_bodies import Sun
from ostk.physics.environment.objects.celestial_bodies import Moon

from ostk.astrodynamics import NumericalSolver
from ostk.astrodynamics.flight import SatelliteSystem
from ostk.astrodynamics.flight.system import SatelliteDynamics
from ostk.astrodynamics.trajectory import State
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Propagated

---

## Propagated Orbit Model

Read in reference data from CSV file 

In [ ]:
filename = 'GMAT_EGM96_24hr_30sInterval_run.csv'
plot_title_extension = 'RMS error for GMAT vs OSTk with EGM96 Force Model at 30s intervals'
legend_name_list = ['OSTk with 30s interval']
# legend_name_list = ['OSTk with tol 1e-15', 'OSTk with tol 1e-15']

multiplication_array = np.array([1.0e3, 1.0e3, 1.0e3])

with open(f'{os.getcwd()}/Validation Files/{filename}', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    reference_state_array = [] 
    instant_array = []
    for row in reader:
        
        instant_iter = start_instant + Duration.seconds(float(row['YAM.ElapsedSecs']))
        position_iter = Position.meters([row['YAM.EarthICRF.X'], row['YAM.EarthICRF.Y'], row['YAM.EarthICRF.Z']], frame_GCRF)
        velocity_iter = Velocity.meters_per_second([row['YAM.EarthICRF.VX'], row['YAM.EarthICRF.VY'], row['YAM.EarthICRF.VZ']], frame_GCRF)
        
        
        position_iter_m = np.multiply(position_iter.get_coordinates(), multiplication_array)
        velocity_iter_ms = np.multiply(velocity_iter.get_coordinates(), multiplication_array)

        
        instant_array.append(instant_iter)
        reference_state_array.append(State(instant_iter, Position.meters(position_iter_m, frame_GCRF), Velocity.meters_per_second(velocity_iter_ms, frame_GCRF)))

### Computation

Create an environment corresponding to the perturbations that should be used

In [ ]:
instant_J2000 = Instant.J2000()
objects = [Earth.EGM96()]

custom_env = Environment(instant_J2000, objects)

Create multiple satellite system, a satellite dynamical system, and a numerical solver

In [ ]:
mass = Mass(90.0, Mass.Unit.Kilogram)
inertia_tensor = np.ndarray(shape=(3, 3))
satellite_geometry = Cuboid(Point(0.0, 0.0, 0.0), [ [1.0, 0.0, 0.0 ], [ 0.0, 1.0, 0.0 ], [ 0.0, 0.0, 1.0 ] ], [1.0, 0.0, 0.0 ] )
surface_area = 0.8
drag_coefficient = 2.2

satellitesystem = SatelliteSystem(mass, inertia_tensor, satellite_geometry, surface_area, drag_coefficient)

start_state = reference_state_array[0]

satellitedynamics = SatelliteDynamics(custom_env, satellitesystem, start_state, SatelliteDynamics.GravitationalPerturbationType.EGM96,
                    SatelliteDynamics.AtmosphericPerturbationType.No, SatelliteDynamics.ThirdBodyPerturbationType.No, SatelliteDynamics.StateVectorDimension.Position_velocity)

numericalsolver_1 = NumericalSolver(NumericalSolver.IntegrationLogType.No_log, NumericalSolver.IntegrationStepperType.Runge_Kutta_Cash_Karp_54, 5.0, 1.0e-15, 1.0e-15)
numericalsolver_2 = NumericalSolver(NumericalSolver.IntegrationLogType.No_log, NumericalSolver.IntegrationStepperType.Runge_Kutta_Fehlberg_78, 5.0, 1.0e-15, 1.0e-15)

Setup a different Propagated models an an instant array at which propagated states are desired

In [ ]:
propagated_model_1 = Propagated(satellitedynamics, numericalsolver_1)
propagated_model_2 = Propagated(satellitedynamics, numericalsolver_2)

propagated_model_list = [propagated_model_1]

Setup the orbit (TBI) to be able to use the LVLH reference frame afterwords 

In [ ]:
# orbit = Orbit(propagated_model, custom_env.access_celestial_object_with_name('Earth'))

Now that everything is set up, we can calculate the state arrays from the desired time instant grid

In [ ]:
# tic
propagated_state_array_list = [[propagated_model_list[ind].calculate_state_at(instant) for instant in instant_array] for ind in range(0,len(propagated_model_list))]
# toc

# orbit_default.get_state_at(current_instant + propagation_duration) 

In [ ]:
def to_dataframe (propagated_state, ind):

    return [
                repr(propagated_state.get_instant()),
                float((propagated_state.get_instant() - propagated_state_array_list[ind][0].get_instant()).in_seconds()),
                *propagated_state.get_position().get_coordinates(),
                *propagated_state.get_velocity().get_coordinates(),
            ]

In [ ]:
propagated_orbit_data_list = [[to_dataframe(propagated_state, ind) for propagated_state in propagated_state_array_list[ind]] for ind in range(0,len(propagated_model_list))]

In [ ]:
propagated_orbit_df_list = [pd.DataFrame(propagated_orbit_data_list[ind], columns=['$Time^{UTC}$', 'Elapsed secs', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']) for ind in range(0,len(propagated_model_list))]

Table:

In [ ]:
[propagated_orbit_df_list[ind].head() for ind in range(0,len(propagated_model_list))]

In [ ]:
[propagated_orbit_df_list[ind].tail() for ind in range(0,len(propagated_model_list))]

#### Trajectory RMS error vs GMAT run in the GCRF frame 

In [ ]:
def to_dataframe_RMS (ref_ind, list_ind):
    
    return [
                repr(propagated_state_array_list[list_ind][ref_ind].get_instant()),
                
                float((propagated_state_array_list[list_ind][ref_ind].get_instant() - propagated_state_array_list[list_ind][0].get_instant()).in_seconds()),

                (np.linalg.norm((propagated_state_array_list[list_ind][ref_ind].get_position().get_coordinates() - reference_state_array[ref_ind].get_position().get_coordinates()))),
                
                # print(f'Prop:{(propagated_state_array[ref_ind].in_frame(frame_LVLH).get_position().get_coordinates())}'),
                # print(f'Ref:{(reference_state_array[ref_ind].in_frame(frame_LVLH).get_position().get_coordinates())}'),
                (np.linalg.norm((propagated_state_array_list[list_ind][ref_ind].get_velocity().get_coordinates() - reference_state_array[ref_ind].get_velocity().get_coordinates()))),
            ]

In [ ]:
orbit_data_RMS_list = [[to_dataframe_RMS(ref_ind, list_ind) for ref_ind in range(0,len(propagated_state_array_list[list_ind]))]  for list_ind in range(0,len(propagated_model_list))]

In [ ]:
orbit_df_RMS_list = [pd.DataFrame(orbit_data_RMS_list[list_ind], columns=['$Time^{UTC}$', 'Elapsed secs', '${\delta}x$', '${\delta}v$'])  for list_ind in range(0,len(propagated_model_list))]

In [ ]:
[orbit_df_RMS_list[list_ind].head() for list_ind in range(0,len(propagated_model_list))]

In [ ]:
[orbit_df_RMS_list[list_ind].tail() for list_ind in range(0,len(propagated_model_list))]

# Validation Plots 

Plot position error 

In [ ]:
orbit_df_RMS_position_list = [orbit_df_RMS_list[list_ind][['Elapsed secs', '${\delta}x$']] for list_ind in range(0,len(propagated_model_list))]

figure = go.Figure()
figure.update_layout(title=f'Position {plot_title_extension}', showlegend = True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Position Difference in RTN (m)")

for list_ind, orbit_df_RMS_position  in enumerate(orbit_df_RMS_position_list):
    figure.add_trace(go.Scatter(
            x = orbit_df_RMS_position['Elapsed secs'],
            y = orbit_df_RMS_position['${\delta}x$'],
            name = legend_name_list[list_ind],
            mode = 'lines'
        ))
    
figure.show()


Plot velocity error

In [ ]:
orbit_df_RMS_velocity_list = [orbit_df_RMS_list[list_ind][['Elapsed secs', '${\delta}v$']] for list_ind in range(0,len(propagated_model_list))]

figure = go.Figure()
figure.update_layout(title=f'Velocity {plot_title_extension}', showlegend = True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Velocity Difference in RTN (m/s)")

for list_ind, orbit_df_RMS_velocity in enumerate(orbit_df_RMS_velocity_list):
    figure.add_trace(go.Scatter(
            x = orbit_df_RMS_velocity['Elapsed secs'],
            y = orbit_df_RMS_velocity['${\delta}v$'],
            name = legend_name_list[list_ind],
            mode = 'lines'
        ))
    
figure.show()
